In [2]:
! pip install smolagents pandas wikipedia langchain langchain-community sentence-transformers datasets rank_bm25 --upgrade -q

Run the following command in your terminal to login:
```
huggingface-cli login
```

# Agentic Retrieval-Augmented-Generation (RAG)

An Agent RAG is better than Vanilla RAG for two reasons:
- It can perform several retrieval steps if the results are not considered satisfactory as in [Self-Query](https://docs.llamaindex.ai/en/stable/examples/evaluation/RetryQuery/)
- It formulates itself a reference sentence as in [HyDE](https://huggingface.co/papers/2212.10496).

## Loading a knowledge base

In [3]:
def extract_wikipedia_pages(page_titles):
    """
    Extracts Wikipedia pages and stores them in a dictionary.

    Args:
        page_titles: A list of Wikipedia page titles to extract.

    Returns:
        A dictionary containing the text of each Wikipedia page.
    """

    page_data = {}
    for title in page_titles:
        try:
            page = wikipedia.page(title)
            content = page.content.strip()
            page_data[page.title] = content
        except wikipedia.exceptions.PageError:
            print(f"Page '{title}' not found.")
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Disambiguation error for '{title}': {e.options}")

    return page_data

In [4]:
import wikipedia

page_titles = [
               "Roger Apéry",
               "Owen Willans Richardson",
               "Otto Sackur",
               "Ludvig Lorenz",
               "Klaus von Klitzing",
               "Henri Victor Regnault",
               "Erwin Madelung",
              ]

# Uncomment the next line to scroll through Wikipedia
# wikipedia_data = extract_wikipedia_pages(page_titles)

Save the dictionary using `json.dump()`:

In [5]:
import json

In [6]:
# with open('wikipedia_data.json', 'w') as f:
#     json.dump(wikipedia_data, f, indent=4)

Load the dictionary using `json.load()`:

In [7]:
with open('wikipedia_data.json', 'r') as f:
    wikipedia_data = json.load(f)

In [8]:
wikipedia_data

{'Roger Apéry': 'Roger Apéry (French: [apeʁi]; 14 November 1916, Rouen – 18 December 1994, Caen) was a Greek-French mathematician most remembered for Apéry\'s theorem, which states that ζ(3) is an irrational number. Here, ζ(s) denotes the Riemann zeta function.== Biography ==Apéry was born in Rouen in 1916 to a French mother and Greek father. His childhood was spent in Lille until 1926, when the family moved to Paris, where he studied at the Lycée Ledru-Rollin and the Lycée Louis-le-Grand.  He was admitted  at the École normale supérieure in 1935.  His studies were interrupted at the start of World War II; he was mobilized in September 1939, taken prisoner of war in June 1940, repatriated with pleurisy in June 1941, and hospitalized until August 1941. He wrote his doctoral thesis in algebraic geometry under the direction of Paul Dubreil and René Garnier in 1947.In 1947 Apéry was appointed Maître de conférences (lecturer) at the University of Rennes. In 1949 he was appointed Professor a

In [9]:
for doc in wikipedia_data:
    print(len(wikipedia_data[doc]))

3113
3461
1801
3363
1841
3431
1487


We use [LangChain](https://python.langchain.com/docs/introduction/) for its vector database utilities.

In [10]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

In [11]:
source_docs = [
    Document(page_content=wikipedia_data[doc], metadata={"source": doc})
    for doc in wikipedia_data
]

In [12]:
source_docs

[Document(metadata={'source': 'Roger Apéry'}, page_content='Roger Apéry (French: [apeʁi]; 14 November 1916, Rouen – 18 December 1994, Caen) was a Greek-French mathematician most remembered for Apéry\'s theorem, which states that ζ(3) is an irrational number. Here, ζ(s) denotes the Riemann zeta function.== Biography ==Apéry was born in Rouen in 1916 to a French mother and Greek father. His childhood was spent in Lille until 1926, when the family moved to Paris, where he studied at the Lycée Ledru-Rollin and the Lycée Louis-le-Grand.  He was admitted  at the École normale supérieure in 1935.  His studies were interrupted at the start of World War II; he was mobilized in September 1939, taken prisoner of war in June 1940, repatriated with pleurisy in June 1941, and hospitalized until August 1941. He wrote his doctoral thesis in algebraic geometry under the direction of Paul Dubreil and René Garnier in 1947.In 1947 Apéry was appointed Maître de conférences (lecturer) at the University of R

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

In [14]:
docs_processed = text_splitter.split_documents(source_docs)
docs_processed[2]

Document(metadata={'source': 'Roger Apéry', 'start_index': 944}, page_content='. In 1949 he was appointed Professor at the University of Caen, where he remained until his retirement.In 1979 he published an unexpected proof of the irrationality of ζ(3), which is the sum of the inverses of the cubes of the positive integers. An indication of the difficulty is that the corresponding problem for other odd powers remains unsolved')

In [15]:
docs_processed

[Document(metadata={'source': 'Roger Apéry', 'start_index': 0}, page_content="Roger Apéry (French: [apeʁi]; 14 November 1916, Rouen – 18 December 1994, Caen) was a Greek-French mathematician most remembered for Apéry's theorem, which states that ζ(3) is an irrational number. Here, ζ(s) denotes the Riemann zeta function.== Biography ==Apéry was born in Rouen in 1916 to a French mother and Greek father. His childhood was spent in Lille until 1926, when the family moved to Paris, where he studied at the Lycée Ledru-Rollin and the Lycée Louis-le-Grand"),
 Document(metadata={'source': 'Roger Apéry', 'start_index': 475}, page_content='.  He was admitted  at the École normale supérieure in 1935.  His studies were interrupted at the start of World War II; he was mobilized in September 1939, taken prisoner of war in June 1940, repatriated with pleurisy in June 1941, and hospitalized until August 1941. He wrote his doctoral thesis in algebraic geometry under the direction of Paul Dubreil and Ren

## The BM25 retriever tool

In [16]:
from smolagents import Tool

class BM25RetrieverTool(Tool):
    name = "BM25_retriever"
    description = "Uses keyword-based search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

BM25retriever_tool = BM25RetrieverTool(docs_processed)

## The embedding retriever tool

In [17]:
import faiss
from sentence_transformers import SentenceTransformer

class EmbeddingRetrieverTool(Tool):
    name = "embedding_retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        model_name = "sentence-transformers/all-mpnet-base-v2"
        self.model = SentenceTransformer(model_name)
        self.docs = docs
        contents = [doc.page_content for doc in docs]
        self.embeddings = self.model.encode(contents)
        dimension = self.embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(self.embeddings.astype('float32'))

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        query_embedding = self.model.encode([query]).astype('float32')
        distances, indices = self.index.search(query_embedding, 5)

        results = []
        for i in indices[0]:
            results.append(self.docs[i])

        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(results)
            ]
        )

embeddingretriever_tool = EmbeddingRetrieverTool(docs_processed)

2025-04-23 11:02:33.898008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745398954.042608    5320 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745398954.084083    5320 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 11:02:34.344611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
ledru_rollin_info = embeddingretriever_tool(query="Information about Roger Apéry")                                    
print(ledru_rollin_info)


Retrieved documents:


===== Document 0 =====
Roger Apéry (French: [apeʁi]; 14 November 1916, Rouen – 18 December 1994, Caen) was a Greek-French mathematician most remembered for Apéry's theorem, which states that ζ(3) is an irrational number. Here, ζ(s) denotes the Riemann zeta function.== Biography ==Apéry was born in Rouen in 1916 to a French mother and Greek father. His childhood was spent in Lille until 1926, when the family moved to Paris, where he studied at the Lycée Ledru-Rollin and the Lycée Louis-le-Grand

===== Document 1 =====
. He abandoned politics after the reforms instituted by Edgar Faure after the 1968 revolt, when he realised that university life was running against the tradition he had always upheld.== Personal life ==Apéry married in 1947 and had three sons, including mathematician François Apéry. His first marriage ended in divorce in 1971. He then remarried in 1972 and divorced in 1977.In 1994, Apéry died from Parkinson's disease after a long illness in Caen

=

## The agent

The agent will need these arguments upon initialization:
- `tools`: a list of tools that the agent will be able to call.
- `model`: the LLM that powers the agent.
Our `model` must be a callable that takes as input a list of messages and returns text. It also needs to accept a stop_sequences argument that indicates when to stop its generation. 

Two options for `model`:
* using HfEngine class provided in the package to get a LLM engine that calls Hugging Face's Inference API.
* using the TransformersModel wrapper

In [19]:
from smolagents import InferenceClientModel, CodeAgent, TransformersModel

# Runs locally, but too small for Agents
# model_name = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
# model = TransformersModel(model_id=model_name)

agent = CodeAgent(
    tools=[BM25retriever_tool, embeddingretriever_tool], 
#     model=model, 
    model=InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct"), 
    max_steps=4,
    additional_authorized_imports=["datetime"],
    verbosity_level=2
)

By curiosity, we can have a look at the default prompt templates:

In [20]:
agent.prompt_templates

{'system_prompt': 'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_ans

## Example run

In [21]:
agent_output = agent.run("Can you list all the date of birth of physicists you know?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you list all the date of birth of physicists you know?                                                      │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To list the dates of birth of physicists, I'll need to search for information on notable physicists and   
extract their dates of birth. I'll start by using the `BM25_retriever` to gather information about some well-known 
physicists.                                                                                                        
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
physicists = ["Albert Einstein", "Isaac Newton", "Marie Curie", "Niels Bohr", "Richard Feynman"]                   
for physicist in physicists:                                                                                       
    info = BM25_retriever(query=f"date of birth of {physicist}")                                                   
    print(f"Date of birth {physicist}: {info}")                                                                    
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  physicists = ["Albert Einstein", "Isaac Newton", "Marie Curie", "Niels Bohr", "Richard Feynman"]                 
  for physicist in physicists:                                                                                     
      info = BM25_retriever(query=f"date of birth of {physicist}")                                                 
      print(f"Date of birth {physicist}: {info}")                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Date of birth Albert Einstein: 
Retrieved documents:


===== Document 0 =====
. In 1949 he was appointed Professor at the University of Caen, where he remained until his retirement.In 1979 he 
published an unexpected proof of the irrationality of ζ(3), which is the sum of the inverses of the cubes of the 
positive integers. An indication of the difficulty is that the corresponding problem for other odd powers remains 
unsolved

===== Document 1 =====
.  In 1840, he was appointed the chair of chemistry of the École Polytechnique, and in 1841, he became a professor 
of physics in the Collège de France.Beginning in 1843, he began compiling extensive numerical tables on the 
properties of steam. These were published in 1847, and inspired Charles Algernon Parsons to develop the steam 
turbine. Regnault received the Rumford Medal of the Royal Society of London and appointment as Chief Engineer of 
Mines

===== Document 2 =====
. In 1851 he was elected a foreign member of the Royal Swedish Academy of Sciences. In 1854 he was appointed 
director of the porcelain works at Sèvres, the Manufacture nationale de Sèvres. In 1855, he was elected to the 
American Philosophical Society.At Sèvres, he continued work on the thermal properties of matter.  He designed 
sensitive thermometers, hygrometers, hypsometers and calorimeters, and measured the specific heats of many 
substances and the coefficient of thermal expansion of gases

===== Document 3 =====
.80745... Ω, is named in honor of Klaus von Klitzing's discovery of the quantum Hall effect, and is listed in the 
National Institute of Standards and Technology Reference on Constants, Units, and Uncertainty. The inverse of the 
constant is equal to half the value of the conductance quantum.More recently, Klitzing's research focuses on the 
properties of low-dimensional electronic systems, typically in low temperatures and in high magnetic fields

===== Document 4 =====
. He continued his studies at the University of Würzburg at the chair of Gottfried Landwehr, completing his PhD 
thesis entitled Galvanomagnetic Properties of Tellurium in Strong Magnetic Fields (German: Galvanomagnetische 
Eigenschaften von Tellur in starken Magnetfeldern) in 1972, and gaining habilitation in 1978
Date of birth Isaac Newton: 
Retrieved documents:


===== Document 0 =====
. In 1949 he was appointed Professor at the University of Caen, where he remained until his retirement.In 1979 he 
published an unexpected proof of the irrationality of ζ(3), which is the sum of the inverses of the cubes of the 
positive integers. An indication of the difficulty is that the corresponding problem for other odd powers remains 
unsolved

===== Document 1 =====
.  In 1840, he was appointed the chair of chemistry of the École Polytechnique, and in 1841, he became a professor 
of physics in the Collège de France.Beginning in 1843, he began compiling extensive numerical tables on the 
properties of steam. These were published in 1847, and inspired Charles Algernon Parsons to develop the steam 
turbine. Regnault received the Rumford Medal of the Royal Society of London and appointment as Chief Engineer of 
Mines

===== Document 2 =====
. In 1851 he was elected a foreign member of the Royal Swedish Academy of Sciences. In 1854 he was appointed 
director of the porcelain works at Sèvres, the Manufacture nationale de Sèvres. In 1855, he was elected to the 
American Philosophical Society.At Sèvres, he continued work on the thermal properties of matter.  He designed 
sensitive thermometers, hygrometers, hypsometers and calorimeters, and measured the specific heats of many 
substances and the coefficient of thermal expansion of gases

===== Document 3 =====
.80745... Ω, is named in honor of Klaus von Klitzing's discovery of the quantum Hall effect, and is listed in the 
National Institute of Standards and Technology Reference on Constants, Units, and Uncertainty. The inverse of the 
constant is equal to half the value of the

...

Execution logs:
Date of birth of Albert Einstein: 
Retrieved documents:


===== Document 0 =====

Date of birth of Isaac Newton: 
Retrieved documents:


===== Document 0 =====

Date of birth of Marie Curie: 
Retrieved documents:


===== Document 0 =====

Date of birth of Niels Bohr: 
Retrieved documents:


===== Document 0 =====

Date of birth of Richard Feynman: 
Retrieved documents:


===== Document 0 =====


Out: 
Retrieved documents:


===== Document 0 =====
.In 1921 he succeeded Max Born as the Chair of Theoretical Physics at the Goethe University Frankfurt, which he 
held until his retirement in 1949. He specialized in atomic physics and quantum mechanics, and it was during this 
time he developed the Madelung equations, an alternative form of the Schrödinger equation

===== Document 1 =====
. He was knighted in 1939.== Works ==The emission of electricity from hot bodies (1st edition, 1916)The emission of
electricity from hot bodies (2nd edition, 1921)                                                                    
== References ==== External links == Media related to Owen Willans Richardson at Wikimedia CommonsOwen Richardson 
on Nobelprize.org

===== Document 2 =====
Erwin Madelung (18 May 1881 – 1 August 1972) was a German physicist.He was born in 1881 in Bonn. His father was the
surgeon Otto Wilhelm Madelung. He earned a doctorate in 1905 from the University of Göttingen, specializing in 
crystal structure, and eventually became a professor. It was during this time he developed the Madelung constant, 
which characterizes the net electrostatic effects of all ions in a crystal lattice, and is used to determine the 
energy of one ion

===== Document 3 =====
Sir Owen Willans Richardson (26 April 1879 – 15 February 1959) was an English physicist who received the Nobel 
Prize in Physics in 1928 for his work on thermionic emission, which led to Richardson's law.== Biography 
==Richardson was born in Dewsbury, Yorkshire, England, the son of Joshua Henry and Charlotte Maria Richardson. He 
was educated at Batley Grammar School and Trinity College, Cambridge, where he gained First Class Honours in 
Natural Sciences

===== Document 4 =====
.== Honours ==Richardson was elected an International Member of the American Philosophical Society in 1910. He was 
elected a Fellow of the Royal Society (FRS) in 1913, and was awarded its Hughes Medal in 1920. He was awarded the 
Nobel Prize in Physics in 1928, "for his work on the thermionic phenomenon and especially for the discovery of the 
law named after him". He was knighted in 1939

[Step 4: Duration 45.41 seconds| Input tokens: 29,785 | Output tokens: 511]

Reached max steps.

[Step 5: Duration 70.93 seconds| Input tokens: 41,426 | Output tokens: 719]

Final output:
Certainly! I will manually provide the correct dates of birth for these physicists:

- **Albert Einstein**: March 14, 1879
- **Isaac Newton**: January 4, 1643 (New Style, or January 25, 1643, Old Style under the Julian calendar)
- **Marie Curie**: November 7, 1867
- **Niels Bohr**: October 7, 1885
- **Richard Feynman**: May 11, 1918

Here is the list:

1. Albert Einstein: March 14, 1879
2. Isaac Newton: January 4, 1643
3. Marie Curie: November 7, 1867
4. Niels Bohr: October 7, 1885
5. Richard Feynman: May 11, 1918
